# 🛰️ 卫星图像分割检测 - 专家改进版
# Satellite Image Segmentation - Expert Improved Version

## 🎯 专家改进要点

基于专家诊断，本版本解决了以下关键问题：

### ❌ 原版本问题
1. **类别权重未生效** - 写在data.yaml中YOLOv8不会解析
2. **权重设置无依据** - 手工指定[1.4,1.2,1.3,0.6]与真实分布不匹配
3. **分辨率不一致** - 训练640验证896导致mAP虚高
4. **Mosaic过度** - 0.8对分割任务有害（边缘撕裂）
5. **学习率策略简单** - 未使用余弦退火

### ✅ 专家改进方案
1. **自动计算类别权重** - 基于有效样本数方法
2. **分割友好增强** - 降低Mosaic，增加Copy-Paste
3. **统一分辨率** - 训练验证推理全程768
4. **余弦退火+AdamW** - 更稳定的收敛
5. **TTA+Tile推理** - 高分辨率卫星图支持

## 1. 🔧 环境设置和依赖安装

In [ ]:
# 安装最新版本的依赖
!pip install -q ultralytics==8.3.3 roboflow matplotlib seaborn pandas scikit-learn

import os
import glob
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from ultralytics import YOLO
from roboflow import Roboflow
from ultralytics.utils.metrics import ConfusionMatrix

print("✅ 环境设置完成!")
print(f"📦 Ultralytics版本: {__import__('ultralytics').__version__}")

## 2. 📥 数据集下载和检查

In [ ]:
# 下载数据集
print("📥 开始下载数据集...")
rf = Roboflow(api_key="EKxSlogyvSMHiOP3MK94")
project = rf.workspace("a-imc4u").project("new-2-6zp4h")
dataset = project.version(1).download("yolov8")

DATA_YAML = os.path.join(dataset.location, "data.yaml")
print(f"✅ 数据集下载完成: {dataset.location}")
print(f"📄 数据配置文件: {DATA_YAML}")

# 检查数据配置
with open(DATA_YAML, 'r') as f:
    data_config = yaml.safe_load(f)

class_names = data_config['names']
num_classes = data_config['nc']
print(f"\n📊 数据集信息:")
print(f"   类别数量: {num_classes}")
print(f"   类别名称: {class_names}")

## 3. 🎯 自动计算类别权重 (专家改进1)

In [ ]:
# 🔍 统计训练集中每个类别的实例数
print("🔍 分析训练集类别分布...")

label_files = glob.glob(os.path.join(dataset.location, 'train/labels', '*.txt'))
counter = Counter()

for f in label_files:
    with open(f) as r:
        for line in r:
            if line.strip():  # 跳过空行
                cls_id = int(line.split()[0])
                counter[cls_id] += 1

print(f"📊 原始类别分布:")
for i in range(num_classes):
    count = counter.get(i, 0)
    print(f"   {class_names[i]:12}: {count:6d} 个实例")

# 🧮 采用"有效样本数"方法计算权重 (Cui et al., 2019)
beta = 0.999  # 重采样参数
freq = np.array([counter.get(i, 0) for i in range(num_classes)], dtype=float)

# 避免除零错误
freq = np.maximum(freq, 1)

eff_num = 1 - np.power(beta, freq)
cls_weights = (1 - beta) / eff_num
cls_weights = cls_weights / cls_weights.mean()  # 归一化

print(f"\n🎯 自动计算的类别权重:")
for i, (name, weight) in enumerate(zip(class_names, cls_weights)):
    print(f"   {name:12}: {weight:.3f}")

print(f"\n💡 权重解释:")
print(f"   - 权重越高 = 该类别越稀少，需要更多关注")
print(f"   - 基于有效样本数方法，比手工设置更科学")
print(f"   - 将直接传入model.train()，不写入data.yaml")

## 4. 🚀 专家改进的模型训练

In [ ]:
# 🎯 专家改进的训练配置
IMG_SIZE = 768  # 统一分辨率，兼顾A100显存与精度

print("🚀 开始专家改进版训练...")
print(f"📊 训练配置:")
print(f"   图像尺寸: {IMG_SIZE}x{IMG_SIZE}")
print(f"   类别权重: 自动计算 (有效样本数方法)")
print(f"   学习率策略: 余弦退火 + AdamW")
print(f"   数据增强: 分割友好 (低Mosaic + Copy-Paste)")

# 加载预训练模型
model = YOLO('yolov8m-seg.pt')
print(f"✅ 预训练模型加载完成")

# 🔧 专家改进的训练参数
training_results = model.train(
    # 基础配置
    data=DATA_YAML,
    epochs=60,                    # 增加训练轮数
    imgsz=IMG_SIZE,              # 统一分辨率
    batch=16,                    # 适中的batch size
    device='auto',               # 自动选择设备
    
    # 🎯 优化器和学习率 (专家改进)
    optimizer='AdamW',           # 对分割任务更稳定
    lr0=2e-4,                   # 更低的初始学习率
    cos_lr=True,                # 余弦退火调度
    warmup_epochs=5,            # 预热轮数
    warmup_momentum=0.8,        # 预热动量
    warmup_bias_lr=0.1,         # 预热偏置学习率
    
    # 🎯 类别权重 (专家改进1)
    class_weights=cls_weights.tolist(),  # 自动计算的权重
    
    # 🎯 分割友好的数据增强 (专家改进2)
    mosaic=0.25,                # 大幅降低Mosaic (原0.8)
    copy_paste=0.5,             # 分割经典增强
    close_mosaic=0,             # 分割任务不延迟关闭
    mixup=0.0,                  # 不使用mixup
    
    # HSV颜色增强
    hsv_h=0.02,                 # 色调变化
    hsv_s=0.7,                  # 饱和度变化
    hsv_v=0.4,                  # 亮度变化
    
    # 几何变换
    degrees=10.0,               # 降低旋转角度
    translate=0.1,              # 平移
    scale=0.5,                  # 缩放
    shear=0.0,                  # 不使用剪切
    perspective=0.0,            # 不使用透视
    flipud=0.5,                 # 垂直翻转
    fliplr=0.5,                 # 水平翻转
    
    # 训练控制
    patience=20,                # 增加早停耐心
    save_period=-1,             # 每epoch自动选择best.pt
    amp=True,                   # 混合精度训练
    
    # 输出设置
    project='runs/segment',
    name='train_expert_v4',
    plots=True,
    save=True
)

print("\n🎉 训练完成!")
print(f"📁 最佳模型: {training_results.best}")